### mlopen kaggle 3 task 

In [180]:
import pandas as pd
import numpy as np

# Загрузка данных

In [181]:
train = pd.read_csv('Downloads/train.csv.gz')
train_checks = pd.read_csv('Downloads/train_checks.csv.gz')

In [182]:
test = pd.read_csv('Downloads/test.csv.gz')
test_checks = pd.read_csv('Downloads/test_checks.csv.gz')

In [183]:
print(train.shape, train_checks.shape)
print(train.columns.values)
print(train_checks.columns.values)

((13682, 5), (2042, 4))
['check_id' 'name' 'category' 'price' 'count']
['check_id' 'shop_name' 'datetime' 'sum']


In [184]:
train.fillna('', inplace=True)

# Подготовка данных

In [185]:
import re
#from transliterate import translit ### did'nt have pip at mac - redo at linux
#todo - выделять отдельный признак из единиц измерения в name

def norm(txt):
  txt = txt.decode('utf-8')
  txt = txt.lower()
#  txt = translit(txt, 'ru', reversed=True)
  txt = re.sub('^[\d\*\:\#]+\s','',txt)
  txt = txt.replace('"','').replace('`','')
  return txt

In [186]:
train['name'] = train['name'].apply(norm)
test['name'] = test['name'].apply(norm)

train.sample(15)

,check_id,name,category,price,count
3366,903,бизнес ланч 4 за 199: картофель фри мал.,Кафе,30.00,1.00
6291,1719,хохланд сыр плавленый фетакса 60 % 200гр,Гастрономия,120.00,1.00
11637,3346,oreo печ.как/нач.вк.ван/шок456г,Чай и сладкое,171.51,1.00
11516,3331,яйцо 1 кат.(с 1)/1дес.,Мясо и птица,39.99,1.00
3096,802,семейные секреты основа для пицц,Гастрономия,49.90,1.00
6429,1861,бананы кг,Овощи и фрукты,54.00,0.73
8989,2493,фольга особ.пч.гриль,Для дома,83.00,1.00
3767,1094,капучино большой,Кафе,99.00,2.00
2066,398,к.ц.шампиньоны резаные ж/б425мл,Овощи и фрукты,72.00,1.00
5523,1424,фосфалюгель 16г №20 сусп 4/20уп,Здоровье,67.36,1.00


In [187]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

vectorizer1 = TfidfVectorizer(max_features=50000, analyzer='char', ngram_range=(1,7))
vectorizer2 = TfidfVectorizer(max_features=12000)

In [188]:
X1 = vectorizer1.fit_transform(train.name)
X2 = vectorizer2.fit_transform(train.name)

In [189]:
import scipy
X = scipy.sparse.hstack([X1, X2])
X.shape

(13682, 62000)

In [190]:
from sklearn.preprocessing import LabelEncoder

labeler = LabelEncoder()

In [191]:
y = labeler.fit_transform(train.category)

# Валидация

In [192]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit

for c in [16]:
    for solver in ['newton-cg']:
        gkf = list(GroupShuffleSplit(n_splits=5, random_state=0).split(X, y, train.check_id.values))
        score = cross_val_score(LogisticRegression(C=c, solver=solver), X, y, scoring='neg_log_loss', cv = gkf, n_jobs = -1)
        print(c, solver, "%.3f +- %.4f" % (-np.mean(score), np.std(score)))

(16, 'newton-cg', '0.416 +- 0.0227')


# Подготовка сабмита

In [193]:
model = LogisticRegression(C=16, solver='newton-cg') # из поперебранного в разделе Валидация
model.fit(X, y)

LogisticRegression(C=16, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False)

In [194]:
X_test1 = vectorizer1.transform(test.name)
X_test2 = vectorizer2.transform(test.name)

X_test1.shape, X_test2.shape

((3000, 50000), (3000, 12000))

In [196]:
X_test = scipy.sparse.hstack([X_test1, X_test2])
X_test.shape

(3000, 62000)

In [197]:
p_test = model.predict_proba(X_test)

In [198]:
test = test[['id']]

In [199]:
for i, c in enumerate(labeler.classes_):
    test[c] = map(lambda x: '%.3f' % x, p_test[:, i])

/Users/administrator/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [200]:
test.to_csv('base_solution.csv.gz', compression='gzip', index = False)